In [1]:
import numpy as np
import pandas as pd

pos_train_data = pd.read_csv('train_pos.csv',sep = ',')
neg_train_data = pd.read_csv('train_neg.csv',sep = ',')
pos_test_data = pd.read_csv('test_pos.csv',sep = ',')
neg_test_data = pd.read_csv('test_neg.csv',sep = ',')

In [2]:
pos_train_data=pos_train_data[['Text','Label']]
neg_train_data=neg_train_data[['Text','Label']]
pos_test_data=pos_test_data[['Text','Label']]
neg_test_data=neg_test_data[['Text','Label']]

In [3]:
data_train=pd.concat([pos_train_data,neg_train_data],ignore_index = True)
data_train = data_train.sample(frac=1).reset_index(drop=True)
data_train.head()

,Text,Label
0,Police Story is one of Jackie Chan's classic f...,1
1,Marlon Brando had long since lost interest in ...,0
2,I am Black American and I loved this soap oper...,1
3,I have never seen a worse movie.<br /><br />It...,0
4,Paul Reiser did a spectacular job in writing t...,1


In [4]:
data_test = pd.concat([pos_test_data,neg_test_data],ignore_index = True)
data_test = data_test.sample(frac=1).reset_index(drop=True)
data_test.head()

,Text,Label
0,I remember seeing this film when I was 13 year...,1
1,Hitchcock once gave an interview where he said...,1
2,A formulaic story with all the tired cliches. ...,0
3,This is one of those films with a great potent...,0
4,I'm not sure I've ever seen a film as bad as t...,0


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [8]:

stop_words=set(stopwords.words('english'))
table = str.maketrans('','',punctuation)

def textclean(text):
    tokens=word_tokenize(text, language="english")
    tokens=[word for word in tokens if word.isalpha()]
    tokens=[w.translate(table) for w in tokens]
    tokens=[word for word in tokens if not word in stop_words]
    token = [word for word in tokens if len(word)>1 ]
    return tokens

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [9]:
reviews=[]
for index,row in data_train.iterrows():
    text=row['Text'].lower()
    reviews.append(textclean(text))

In [10]:
reviews[0]

['police',
 'story',
 'one',
 'jackie',
 'chan',
 'classic',
 'films',
 'helped',
 'shape',
 'hong',
 'kong',
 'cinema',
 'masterpiece',
 'missed',
 'action',
 'movie',
 'fan',
 'beginning',
 'obvious',
 'jackie',
 'chan',
 'stunt',
 'team',
 'literally',
 'risked',
 'live',
 'make',
 'film',
 'action',
 'stunts',
 'extremely',
 'realistic',
 'innovative',
 'even',
 'today',
 'movie',
 'outdone',
 'police',
 'story',
 'dangerous',
 'stunts',
 'many',
 'people',
 'hospitalized',
 'police',
 'story',
 'including',
 'jackie',
 'chan',
 'fighting',
 'indisputably',
 'exceptional',
 'stunts',
 'fighting',
 'movie',
 'helped',
 'change',
 'define',
 'jackie',
 'chan',
 'use',
 'props',
 'throughout',
 'film',
 'chan',
 'uses',
 'odd',
 'object',
 'stop',
 'attackers',
 'constantly',
 'throwing',
 'assailants',
 'thick',
 'glass',
 'action',
 'feels',
 'real',
 'stuntmen',
 'giving',
 'movie',
 'give',
 'jackie',
 'chan',
 'coordination',
 'br',
 'br',
 'rest',
 'aspects',
 'film',
 'without'

In [24]:
#pip install -U gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24
Note: you may need to restart the kernel to use updated packages.


In [11]:
#pip install word2vec

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=181934 sha256=be6cbdea1ba09cf9ce9bd256650c0369bdb6f3812072c52965adfece06c1a27e
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\d7\3e\81\4f4daaa6d86f5b9301c16cb1051baa2cbf8f0a309063862b2b
Successfully built word2vec
Note: you may need to restart the kernel to use updated packages.


In [11]:
import gensim
from gensim.models import Word2Vec
n_dim=200

w2v_model =Word2Vec(reviews,min_count=5,vector_size=200)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer  = TfidfVectorizer(min_df=10,analyzer=lambda x: x)

matrix=vectorizer.fit_transform(reviews)



In [13]:
tfidf = dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))


In [14]:
def create_word_vector(l,size):
    vector = np.zeros(size).reshape((1,size))
    count = 0.
    for word in l:
        try:
            vector += w2v_model.wv[word].reshape((1, size)) * tfidf[word]
            count+=1
        except KeyError:
            continue
            
    if count!=0:
        vector /= count
    return vector

In [15]:
X_train = []
y_train = []

for i in range(len(data_train)):
    converted_review = create_word_vector(reviews[i],n_dim)
    #print(converted_review)
    X_train.append(np.array(converted_review))
    y_train.append(data_train['Label'][i])

In [16]:
from sklearn.preprocessing import scale

X_train = np.concatenate(X_train)
X_train = scale(X_train)
y_train = np.array(y_train)


In [17]:
X_train.shape

(25000, 200)

In [18]:
data_test = pd.concat([pos_test_data,neg_test_data],ignore_index = True)
data_test = data_test.sample(frac=1).reset_index(drop=True)

validation_reviews = []

for index,row in data_test.iterrows():
    text = (row['Text'].lower())
    validation_reviews.append(textclean(text))
    
X_val = []
y_val = []

for i in range(len(data_test)):
    converted_review = create_word_vector(validation_reviews[i],n_dim)
    X_val.append(converted_review)
    y_val.append(data_test['Label'][i])
        
X_val = np.concatenate(X_val)
X_val = scale(X_val)
y_val = np.array(y_val)

In [19]:
X_val.shape

(25000, 200)

In [20]:
from sklearn.svm import SVC

clf = SVC()
clf = clf.fit(X_train,y_train)

In [21]:
print(clf.predict(X_val[4].reshape(1,-1)))

[0]


In [22]:
data_test['Text'][4]

"This movie sucked. The problem was not with the cast. I think the cast was great, lots of good talent, lots of great acting. But the script was TERRIBLE! It seemed to be mostly just a frame work in which Steve Carrell could do his improv. And that is what he does best, but it just didn't work here. The script was hard to follow, the story was non-sensical, and scenes were random and lacked direction. Also, much of the action was extremely contrived and poorly thought out. It was a good effort, but as Max says, they missed it by THAT MUCH! I am shocked to see how many glowing reviews there are for this stinker here on the IMDb. Obviously, the movie producers are getting people to write lots of positive reviews on their movies and fill up the entries on the IMDb. If you read the positive reviews and compare them to the negative reviews, it is pretty clear which ones are genuine reviews from normal users.<br /><br />This movie was full of problems and jokes that just didn't work. I loved

In [23]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val,clf.predict(X_val)))

0.85456
